<a href="https://colab.research.google.com/github/jtlai0921/-/blob/master/keras_04transferlearning_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import keras
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten
from keras.models import Model
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [3]:
IMG_SIZE = 128
class_map = {
    'PS':0,
    'Sc':1,
    'RS':2,
    'In':3,
    'Cr':4,
    'Pa':5
}

In [5]:
# Read single image
data_path = 'drive/My Drive/Day2/NEU_defect/*.bmp' 
img_paths = glob(data_path)
img_paths = np.random.choice(img_paths, 300) # demo for 300 images
img = cv2.imread(img_paths[0])
plt.imshow(img)

ValueError: ignored

In [7]:
# Resize img
img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
plt.imshow(img_resized)
img.shape, img_resized.shape

NameError: ignored

In [8]:
# Extract img class
img_path = img_paths[0]
img_path

IndexError: ignored

In [10]:
cls = img_path.split('/')[-1].split('_')[0]
cls

NameError: ignored

In [11]:
class_map[cls]

NameError: ignored

In [ ]:
# method 1: Load Image and labels
data_count = len(img_paths)
X = np.zeros((data_count, IMG_SIZE, IMG_SIZE, 3))
y = np.zeros((data_count, ))

In [12]:
for i, path in tqdm(enumerate(img_paths)):
    img = cv2.imread(path)
    img_resize = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    cls = path.split('/')[-1].split('_')[0]
    img_resize = img_resize/255
    X[i] = img_resize
    y[i] = class_map[cls]

In [13]:
# Method2: load Images and labels from pickle file
import pickle
with open('/content/drive/My Drive/Day2/X.pickle', 'rb') as file:
    X = pickle.load(file)
with open('/content/drive/My Drive/Day2/y.pickle', 'rb') as file:
    y = pickle.load(file)
X.shape, y.shape

FileNotFoundError: ignored

In [14]:
# label count
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

NameError: ignored

In [ ]:

y_onehot = keras.utils.to_categorical(y, num_classes=6)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=5566)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

NameError: ignored

In [ ]:
# Use InceptionV3 as base model
base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)
model = Model(base_model.input, predictions)

In [ ]:
# freeze layers
base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)
model = Model(base_model.input, predictions)

for layer in base_model.layers[:-10]:
    layer.trainable = False


In [ ]:
model.summary()

In [16]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

NameError: ignored

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

checkpoint = ModelCheckpoint('drive/My Drive/ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-6)


In [17]:

logs = model.fit(X_train, y_train,
                batch_size=32,
                epochs=100,
                verbose=1,
                callbacks=[early_stopping, reduce_lr],     
                validation_data=(X_test, y_test))

NameError: ignored

In [ ]:
history = logs.history

In [18]:
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.title('acc')

NameError: ignored

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('loss')

In [19]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = np.argmax(y_test, axis=-1)
y_pred = np.argmax(model.predict(X_test), axis=-1)
print(y_true.shape, y_pred.shape)
target_names = [str(i) for i in range(3)]
print(classification_report(y_true, y_pred, target_names=target_names))
print(confusion_matrix(y_true, y_pred))


NameError: ignored